# Classical Monte Carlo of $S_N$ Model on Hypertree Lattice

## Statistical Mechanics

### Generic Model
The most general StatMech model we consider is specified by the following ingredients:
* **Lattice**: we will only consider *bipartite* lattice, on which the block Gibbs sampling can be carried out.
 * The bipartite lattice contains two **sublattices** labeled by *A* and *B*.
 * The **boundary**, labeled by *C*, refers to the sites that will not be updated in Monte Carlo. The boundary configuration is fixed by the boundary condition and will not be updated. 
* **Group**: on each site, the degree of freedom is a permutation group element $g_i\in S_n$, which we will called the "spin" hereinafter.
 * Each spin state will be represented as a vector in the canonical representation of $S_n$. For example, for $S_2$ group: $|()\rangle=(1,0)^\intercal$, $|(12)\rangle=(0,1)^\intercal.$
* **Hamiltonian**:
$$H[g] = -\sum_{\langle ij\rangle} K_{ij}\chi(g_i,g_j),$$
where $\chi(g_i,g_j)=\text{Tr}g_i^{-1}g_j$ is the group character (the number of cycles). The partition function is given by
$$Z = -\sum_{[g|A\cup B]}e^{-H[g]} = e^{-F[g|C]}.$$
With given boundary, the free energy $F$ is a function of the boundary configuration.

### Monte Carlo Algorithm
The basic idea of importance sampling is to set up a suitable Markov chain that draws configurations according to the Boltzmann weight
$$P[g] = \frac{e^{-H[g]}}{Z}.$$
There are two classes of update schemes:
* Local update algorithms (Metropolis, **Heat-Bath**)
* Cluster update algorithms (Swendsen-Wang Cluster, Wolff Cluster)

The cluster update greately reduces the dynamic scaling at the critical point, which speed up the convergence in the critical region. Cluster update works well for Potts model, where the spins interacts via delta function: such that the domain wall between all different spins are of the same tension (energy). In the Wolff cluster algorithm, a site is first choosen, and neighboring sites of the same spin can attach to the cluster with probability $p$ (sites of different spins should not attach), and finally the whole cluster is flipped. The transition rate can satisfy the detail balance because the intereial weight is the same ($=p^{C-1}$ where $C$ is number of cluster site, this is because the cluster is grown following a tree, and all trees are equivalent in bond number if the site number is fixed), so the only difference is on the boundary. There are two types of cluster boundaries: boudaries between different spins (of tension 0) and boundary between same spins (of tension $-\ln p$). After flipping, the two types of boundary will switch their positions, so it can match the detail balance condition. More precisely, the partition function for $n$-state Potts model can be cast into
$$Z\propto\sum_{\text{config}}p^{\text{#link}}n^{\text{#cluster}}.$$
This is due to the nice delta function structure. However for our $S_n$ model, the delta interaction is soften in some sense. Then the partition function no longer has the nice link and cluster structure, which make it hard to do cluster update.

While it remains a question of <font color='red'>how to design a cluster update algorithm for $S_n$ model</font>, we will try the local update first, in particular the heat-bath update. The current code uses **block Gibbs sampling** + **heat-bath update**, alternatively sampling on A and B sublattices.

Question: <font color='red'>How to determine equilibrium?</font>

### Temperature Profiling

Question: <font color='red'>How to calculate free energy?</font>

## Code Structure
Objects:
* **Group**
* **Lattice** (HypertreeLattice)
* **Model** (LatticeModel)
    * Wrapper of the FORTRAN extension MC (provides interface for data transfer and control).

In [10]:
%reload_ext snakeviz
%run 'MC.py'

### <code>Group</code> Object

`Group(n)` represents a permutation group $S_n$.
* Each element is a tuple of $n$ numbers, representing the permutation of $(0,1,...,n-1)$.
* `Group.dof` gives the order of the group, i.e. $n!$, which is also the degrees of freedom on each site.
* `Group.chi` the table of $\chi(g_1,g_2)$ (adjusted by $-n$).
* `Group.element` gives the list of all group elements.
* `Group.index` is a dictionary that maps group element to its index.

After initialization, `Group.dof` and `Group.chi` should have been set.

In [2]:
'''multiplication table'''
group = Group(3)
[[group.index[group.multiply(g,h)] for h in group] for g in group]

[[0, 1, 2, 3, 4, 5],
 [1, 0, 4, 5, 2, 3],
 [2, 3, 0, 1, 5, 4],
 [3, 2, 5, 4, 0, 1],
 [4, 5, 1, 0, 3, 2],
 [5, 4, 3, 2, 1, 0]]

In [8]:
'''check group inversion'''
[group.index[group.multiply(g, group.inverse(g))] for g in group]

[0, 0, 0, 0, 0, 0]

In [3]:
'''characters'''
{g: group.character(g) for g in group}

{(0, 1, 2): 3,
 (0, 2, 1): 2,
 (1, 0, 2): 2,
 (1, 2, 0): 1,
 (2, 0, 1): 1,
 (2, 1, 0): 2}

The character of a permutation is defined as the number of cycles in the permutation, denoted as $\ln\text{Tr}g$, ranging from $1$ to $n$. Upon initialization, the <code>Group</code> object generates the table of
$$\chi(g_1,g_2)=\ln\text{Tr} g_1 g_2^{-1}-n.$$
One can show that $\chi(g_1,g_2)=\chi(g_2,g_1)$ and $\forall g: \chi(gg_1,gg_2)=\chi(g_1g,g_2g)=\chi(g_1,g_2)$.
* This is the interaction function between two spins across a bond.
* The shift $-n$ is needed to avoid partition weight overflow at low temperature. 

In [4]:
Group(3).chi

[[0, -1, -1, -2, -2, -1],
 [-1, 0, -2, -1, -1, -2],
 [-1, -2, 0, -1, -1, -2],
 [-2, -1, -1, 0, -2, -1],
 [-2, -1, -1, -2, 0, -1],
 [-1, -2, -2, -1, -1, 0]]

### <code>Lattice</code> Object

Site arranging rules: AB must before C, because will not allocate for C, contineous space for AB, within each sublattice dont scatter around.

In [2]:
HypertreeLattice(4)

[<2,0,0>, <2,1,0>, <2,2,0>, <2,3,0>, <1,0,0>, <1,0,1>, <1,1,0>, <1,1,1>, <0,0,0>, <0,0,1>, <0,0,2>, <0,0,3>]

### <code>Model</code> Object

The CPU intensive Monte Carlo samplings are performed by the FORTRAN extension MC. The module MC contains two parts:
* core (for MC sampling and update macroscopic observibles)
* physics (for data collection and measurements)

The Model object is a python wapper for the MC module.

In [5]:
print(MC.__doc__)

This module 'MC' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  core --- nc,nlst,nsite,chi,irng,nb,dof,klst,energy,config,na,jlst,hist,init(),run(),get_energy(),get_hist(),dump(),load()  physics --- magnet2,nspin,monitor,energy1,energy2,spins,magnet1,measure().


#### Attributes

The three-block data structure: system, state and data.
- **`system`**: static, not changed during MC update, defines the system. 
  - The full set of system parameters are specified by the Model class attribute `Model.system_parameters`
  - During the temperature ramping, only `klst` will change. The other parameters are fixed by the lattice graph and on-site group.
- **`state`**: dynamic, updated by MC sampling.
  - Can be kept for future restart of the Markov chain.
- **`data`**: disposible, generated by the `measure` method.

Mapping the attributes between **MC** (**FORTRAN** module) and **Model** (**python** object):
- The system and state parameters are linked (as inout) between MC and Model.
- The data parameters are output from MC to Model after each measurement.


    MC                                                        Model
    +- MC.core                                   Model.system[:] -+
    |  +- MC.core.dof                   <==> Model.dof        -+  |
    |  +- MC.core.nsite                 <==> Model.nsite      -+  |
    |  +- MC.core.na                    <==> Model.na         -+  |
    |  +- MC.core.nb                    <==> Model.nb         -+  |
    |  +- MC.core.nc                    <==> Model.nc         -+  |
    |  +- MC.core.nlst                  <==> Model.nlst       -+  |
    |  +- MC.core.chi[:dof,:dof]        <==> Model.chi        -+  |
    |  +- MC.core.irng[:nsite+1]        <==> Model.irng       -+  |
    |  +- MC.core.jlst[:nlst]           <==> Model.jlst       -+  |
    |  +- MC.core.klst[:nlst]           <==> Model.klst       -+  |
    |  |                                          Model.state[:] -+
    |  +- MC.core.config[:nsite]        <==> Model.config     -+  |
    |  +- MC.core.energy                <==> Model.energy     -+  |
    |  +- MC.core.hist[:dof]            <==> Model.hist       -+  |
    +- MC.physics                                                 |
       +- MC.physics.nspin                                        |
       +- MC.physics.monitor[:nspin]               Model.data[:] -+
       +- MC.physics.energy1            ===> Model.data[energy1]
       +- MC.physics.energy2            ===> Model.data[energy2]
       +- MC.physics.magnet1[:dof]      ===> Model.data[magnet1]
       +- MC.physics.magnet2[:dof,:dof] ===> Model.data[magnet2]
       +- MC.physics.spins[:dof,:nspin] ===> Model.data[spins]
                                             Model.data[steps]       

Notes:
- The `Model` attributes *do not physically exist*. They are accessed by `getattr` and `setattr` methods. The system and state attributes can be set as a whole by passing the dictionary. The data attributs are read only, setting `Model.data` will not affect the variables in the MC module.
- `Model` object provides **state parameter parsing**.
  - `config`: 'FM', 'uniform' = initialize with 0's; 'PM', 'random' = initialize with random spins from `range(dof)`.
  - `energy`: 'unknown', 'nan' = set energy to nan (which is a flag telling MC that the energy has not be calculated yet).
  - `hist`: 'unknown' = set hist to empty with hist[0]=-1 (which is a flag telling MC that the histogram has not been collected yet).
- When attribute getter is called to get `energy` and `hist`, it is first checked whether they have been calculated, if not, will evoke MC module (by calling `MC.core.get_energy()` or `.get_hist()`) to calculate and then return.

#### Methods

`Model.run(steps, mode)` runs the MC updates for specific `steps`, under the `mode`:
- `mode = 0` (default): update without keeping track of the `energy` and `hist`.
- `mode = 1`: `energy` and `hist` are updated with the MC run at every spin flip.

`Model.measure(step, monitor)` runs MC updates for specific `steps`, and returns the measurement on the samples of:
- energy 1st and 2nd moment: $\langle E\rangle$, $\langle E^2\rangle$
- total magnetization 1st and 2nd moment: $\langle M\rangle$, $\langle M^2\rangle$ (where $M=N_{A\cup B}^{-1}\sum_{i\in A\cup B} g_i$ and $g_i$ is represented as a vector in the canonical representation).
  - the 2nd moment $\langle M^2\rangle$ is a matrix, containing infomations of cross correlation.
- specific spins (averged over samples), spin indices are specified by <code>monitor</code>, and the number of spin to monitor is specified by <code>nspin</code>.
- the number of steps (samples) is also returned, such that the data can be combined with previous measurements properly, as weighted by the number of samples.

#### Core Performance

**Heat-Bath** update: on each site $i$, randomly draw a new spin state $g_i$ (regardless of the previous spin state) with probability:
$$p(g_i)=\frac{e^{h(g_i)}}{\sum_{g'_i}e^{h(g'_i)}}, h(g_i)=\sum_{j \text{n.n.}i}K_{ij}\chi(g_i,g_j).$$

**Block-Gibbs** sampling: spins in the same sublattice are block-updated.
- For A sublattice: `I=1:NA`
- For B sublattice: `I=NA+1:NA+NB`.

FROTRAN implementation (for each spin update):
- `RAND_NUMBER`: initialize `RND` by a random real drawn from the uniform distribution in $[0,1]$. gfortran implements KISS random number generator, $2^{123}$.
- `SET_BLOCK`: sets the cumulated weights and renoralized random real
  - `SET_BIAS`: calculate the on-site bias fields (site indexed by `I`) as `sum(CHI[:,CONFIG[JLST[J]]]*KLST[J] for J in range(IRNG[I],IRNG[I+1]))`. The result is a vector of `DOF` components, which specify the bias field for all group elements.
  - `SET_WEIGHT`: exponentiate the bias fields and accumulate to unormalized CDF weights. The time of this step can vary from 4ns to 10ns depending on the value of the exponent. `EXP(x)` has a quick return for `x=0` but slower for generic `x`. The last compunent is the total partition weight.
  - `RND *= WEIGHT[-1]`: the random real is then multiplied by the total partition weight, such that it is distributed uniformly from 0 to the total weight.
- `CHOOSE`: `WEIGHT` and `RND` is then used to make choice of the new spin state, implemented by a binary serach to determine which bin in `WEIGHT` does `RND` fall into.

In each step (sweep) of MC sampling, the *entire bulk is updated* (which contains two block Gibbs sampling: A-given-B and B-given-A). Measurements are made after the bulk update, so the measurement time is neglegable. For one MC step, the complexity depends on:
- $N$ - the number of bulk sites.
- $n!$ - group order (onsite degrees of freedom).

<table align="left">
<tr><td>procedure</td><td>complexity</td><td>time (ns)</td></tr>
<tr><td><code>RAND_NUMBER</code></td><td>$N$</td><td>13.8</td></tr>
<tr><td><code>SET_BLOCK</code></td><td>$Nn!$</td><td>12.1</td></tr>
<tr><td><code>    SET_BIAS</code></td><td>$Nn!$</td><td>2.3</td></tr>
<tr><td><code>    SET_WEIGHT</code></td><td>$Nn!$</td><td>9.8</td></tr>
<tr><td><code>        EXP</code></td><td>$Nn!$</td><td>8.0</td></tr>
<tr><td><code>    RND</code></td><td>$N$</td><td>2.1</td></tr>
<tr><td><code>CHOOSE</code></td><td>$N\log(n!)$</td><td>5.6</td></tr>
</table>

Average update time per spin scales with $n$ as:
<table align="left">
<tr><td>$n=$</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td></tr>
<tr><td>time(μs)</td><td>0.056</td><td>0.12</td><td>0.35</td><td>1.52</td><td>9.13</td></tr>
</table>

## Hypertree Model

### Example
Generate model, run, get hist and energy, make measurements.
- use **block Gibbs sampling** + **heatbath update**, alternatively sampling on A and B sublattices.
  - A sublattice is the one that contains the most UV layer
  - B sublattice contains the rest
  - the boundary is labeled by C
- energy is calculated for all sites including the boundary spins.
- hist and magnetization only conts the bulk spins, boundary spins are not taken into account.

In [2]:
sys = LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(3)).run(100)
for k in range(10):
    sys.run(mode = 1)
    print(sys.config,sys.hist,sys.energy)
sys.measure(1000,monitor=[0,1,2,3])

[0 2 0 1 0 0 5 1 0 0 0 0] [4 2 1 0 0 1] 5.199999999999999
[0 3 3 3 0 5 1 3 0 0 0 0] [2 1 0 4 0 1] 7.599999999999999
[4 1 3 1 1 0 2 1 0 0 0 0] [1 4 1 1 1 0] 9.999999999999998
[0 2 0 3 0 0 2 0 0 0 0 0] [5 0 2 1 0 0] 3.9999999999999987
[2 0 0 3 0 5 2 2 0 0 0 0] [3 0 3 1 0 1] 9.999999999999996
[0 5 2 2 0 0 3 2 0 0 0 0] [3 0 3 1 0 1] 4.399999999999997
[0 3 2 3 0 0 2 2 0 0 0 0] [3 0 3 2 0 0] 3.5999999999999974
[1 2 2 4 0 0 0 0 0 0 0 0] [4 1 2 0 1 0] 6.799999999999999
[1 2 0 1 4 0 0 0 0 0 0 0] [4 2 1 0 1 0] 8.399999999999999
[0 5 0 5 0 1 5 0 0 0 0 0] [4 1 0 0 0 3] 6.799999999999997


{'energy1': 6.492800000000001,
 'energy2': 47.164479999999934,
 'magnet1': array([ 0.416   ,  0.12875 ,  0.141375,  0.08725 ,  0.0875  ,  0.139125]),
 'magnet2': array([[ 0.20721875,  0.04709375,  0.05221875,  0.03079688,  0.0299375 ,
          0.04873438],
        [ 0.04709375,  0.0350625 ,  0.01214063,  0.00970312,  0.009875  ,
          0.014875  ],
        [ 0.05221875,  0.01214063,  0.04076562,  0.01082812,  0.01059375,
          0.01482812],
        [ 0.03079688,  0.00970312,  0.01082812,  0.01996875,  0.0056875 ,
          0.01026563],
        [ 0.0299375 ,  0.009875  ,  0.01059375,  0.0056875 ,  0.02109375,
          0.0103125 ],
        [ 0.04873438,  0.014875  ,  0.01482812,  0.01026563,  0.0103125 ,
          0.04010938]]),
 'spins': array([[ 0.578,  0.252,  0.234,  0.166],
        [ 0.092,  0.162,  0.152,  0.172],
        [ 0.124,  0.167,  0.179,  0.172],
        [ 0.033,  0.127,  0.14 ,  0.162],
        [ 0.037,  0.118,  0.136,  0.161],
        [ 0.136,  0.174,  0.159,  0.

### Profiling

In [5]:
%reload_ext snakeviz
%run 'MC.py'
%snakeviz LatticeModel(HypertreeLattice(128,[1.,1.]),Group(6)).run(10000,0)

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmpg6at70en'. 
